In [ ]:
import os
from os import path
from astropy.time import Time
from astropy.io import fits, ascii
import astropy.units as u
from astropy.table import Table
from astropy.constants import G

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import h5py
from sqlalchemy import func
from scipy.optimize import root
from scipy.stats import scoreatpercentile
import tqdm

from thejoker import JokerSamples
from thejoker.sampler import JokerParams, TheJoker
from thejoker.plot import plot_rv_curves

from twoface.config import TWOFACE_CACHE_PATH
from twoface.db import (db_connect, AllStar, AllVisit, AllVisitToAllStar, RedClump,
                        StarResult, Status, JokerRun, initialize_db)
from twoface.data import APOGEERVData
from twoface.plot import plot_data_orbits

In [ ]:
TWOFACE_CACHE_PATH = path.abspath('../cache/')
samples_file = path.join(TWOFACE_CACHE_PATH, 'apogee-jitter.hdf5')

In [ ]:
Session, _ = db_connect(path.join(TWOFACE_CACHE_PATH, 'apogee.sqlite'))
session = Session()

In [ ]:
stars = session.query(AllStar).join(StarResult, Status, AllVisitToAllStar, AllVisit)\
                              .filter(Status.id > 0)\
                              .filter(AllStar.logg > 0)\
                              .group_by(AllStar.apstar_id)\
                              .having(func.count(AllVisit.id) >= 12)\
                              .all()
len(stars)

In [ ]:
%%time

rows = None
with h5py.File(samples_file, 'r') as f:
    for star in stars:
        samples = JokerSamples.from_hdf5(f[star.apogee_id])
        loggs = [star.logg] * len(samples)
        
        this_rows = list(zip(loggs, 
                             samples['P'].to(u.day).value,
                             samples['e'].value))
        
        if rows is None:
            rows = this_rows
        else:
            rows = rows + this_rows
            
tbl = np.array(rows, dtype=[('logg', float), ('P', float), ('e', float)])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.plot(tbl['P'], tbl['logg'], marker='.', 
        linestyle='none', alpha=0.1, ms=2)

# -- P curves --
logg = np.linspace(0, 4, 1024)
g = 10**logg * u.cm/u.s**2

for M1 in [0.5, 1., 2] * u.Msun:
    R = np.sqrt(G * M1 / g).to(u.au)

    P = (2*np.pi * np.sqrt(R**3 / (G * M1))).to(u.day).value
    ax.plot(P, logg, marker='', 
            label='{0:.1f} {1:latex_inline}'.format(M1.value, M1.unit))

ax.legend(loc='upper right')
    
ax.set_xscale('log')
ax.set_xlim(1, 32768)
ax.set_ylim(3.55, 0)
ax.set_xlabel('$P$ [day]')
ax.set_ylabel('$\log g$')
    
fig.tight_layout()

In [ ]:
stars2 = session.query(AllStar).join(StarResult, Status)\
                               .filter(Status.id == 2).all()
len(stars2)

In [ ]:
%%time

rows = None
with h5py.File(samples_file, 'r') as f:
    for star in stars2:
        samples = JokerSamples.from_hdf5(f[star.apogee_id])
        loggs = [star.logg] * len(samples)
        
        this_rows = list(zip(loggs, 
                             samples['P'].to(u.day).value,
                             samples['e'].value))
        
        if rows is None:
            rows = this_rows
        else:
            rows = rows + this_rows
            
tbl2 = np.array(rows, dtype=[('logg', float), ('P', float), ('e', float)])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.plot(tbl2['P'], tbl2['logg'], marker='.', 
        linestyle='none', alpha=0.4, ms=6)

# -- P curves --
logg = np.linspace(0, 4, 1024)
g = 10**logg * u.cm/u.s**2

for M1 in [0.5, 1., 2] * u.Msun:
    R = np.sqrt(G * M1 / g).to(u.au)

    P = (2*np.pi * np.sqrt(R**3 / (G * M1))).to(u.day).value
    ax.plot(P, logg, marker='', 
            label='{0:.1f} {1:latex_inline}'.format(M1.value, M1.unit))

ax.legend(loc='upper right')
    
ax.set_xscale('log')
ax.set_xlim(1, 32768)
ax.set_ylim(3.55, 0)
ax.set_xlabel('$P$ [day]')
ax.set_ylabel('$\log g$')
    
fig.tight_layout()